**TODO**
- get stats for droughted area (convex hull):
 - spatial join convex hull with original grid with PPET values
 - get values for all 9km x 9km squares in hull
 - do proportional split on squares that don't lie 100% in convex hull
 - get area
- get days over time period specified
- get centroid for hull

In [ ]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

import requests as rq

# for aWhere API
from header import AWhereAPI
from secret_codes import *

api_key = API_KEY
api_secret = API_SECRET

In [ ]:
# Create aWhere object
aw = AWhereAPI(api_key, api_secret)

# To get auth token, encode secret and key
sc = aw.encode_secret_and_key(API_KEY, API_SECRET)

# Then call auth token
token = aw.get_oauth_token(sc)

In [ ]:
""" Set variables for aWhere API call"""

# General endpoint
endpt = "https://api.awhere.com"

# Set up headers: auth and specific header for agronomics data
auth_headers = {"Authorization": "Bearer %s" % token,
                "Content-Type": 'application/json'}

agro_header = {"Authorization":  "Bearer %s" % token}

In [ ]:
# Create path to home
home = str(Path.home())

# Create path to geojson since csv crashes Jupyter
path_to_marin = os.path.join(
    home, "Desktop", "drought-tracker", "data", "TIGER", "marin.geojson")

# Create df with 1 test county (Marin)
marin = gpd.read_file(path_to_marin)

marin = marin.to_crs("EPSG:4326")

In [ ]:
def create_awhere_grid(aoi, out_crs, calc_crs):
    """
    Parameters
    ----------
    aoi: Geopandas GeoDataFrame
        One-element GeoDataFrame containing area over which
        to draw grid
    
    out_crs: numeric string
        String of EPSG code for exported GDF
    
    calc_crs: numeric string
        String of EPSG code for CRS used
        to calculate grid. CRS must have units of meters
        
    Returns
    -------
    GeoDataFrame containing 9km x 9km cells
    """   
    
    # Reproject aoi to CRS using meters for aWhere grid cells
    aoi = aoi.to_crs(f'epsg:{calc_crs}')
    
    # Get x and y min and max from total boundaries
    xmin, ymin, xmax, ymax = aoi.total_bounds
    
    # Set side of grid cell to 9 km
    side = 9000
    
    # Create x values for x points for rows
    x_range = np.arange(int(np.floor(xmin)), int(np.ceil(xmax)), side)
    
    # Create y values
    y_range = np.arange(int(np.floor(ymin)), int(np.floor(ymax)), side)
    
    # Create empty list to hold grid cells
    polygons = []
    
    for x in x_range:
        for y in y_range:
            polygons.append(Polygon([(x,y), (x+side, y), (x+side, y+side), (x,y+side)]))
            
    grid = gpd.GeoDataFrame({'geometry': polygons})
    
    grid.crs = f'EPSG:{calc_crs}'
    
    # Convert to out_crs
    grid = grid.to_crs(f"EPSG:{out_crs}")
    
    return grid

In [ ]:
# Create grid polygon
grid = create_awhere_grid(aoi=marin,
                          out_crs='4326',
                          calc_crs='2019')

# Add centroid column
grid['centroid'] = grid.geometry.apply(lambda poly: poly.centroid)


# Check to see if everything looks correct
fig, ax = plt.subplots()
grid.plot(color="none", linewidth=0.2, edgecolor='red', ax=ax)
marin.plot(color='steelblue', alpha=0.4, ax=ax)

In [ ]:
for index, row in grid.iterrows():
    print(round(row.centroid.x, 5))

In [ ]:
# URL to historical agronomics (norms)
hist_ag_url = f"/v2/agronomics/locations/40,-105/agronomicvalues/2020-04-01,2020-05-30"

# Endpoint suffix to get all accumulations
url_append = "?properties=accumulations"

full_url = endpt + hist_ag_url + url_append

ag_norms = rq.get(full_url, headers=agro_header)

ag_norms_json = ag_norms.json()

In [ ]:
def get_ppet(grid_df, start_date, end_date):

    if 'ppet' not in grid_df.columns:
        grid_df = grid_df.assign(ppet=0)

    for index, row in grid_df.iterrows():

        # Get centroid coordinates from grid_df
        lat = round(row.centroid.y, 5)
        lon = round(row.centroid.x, 5)

        # URL to historical agronomics (norms)
        hist_ag_url = f"/v2/agronomics/locations/{lat},{lon}/agronomicvalues/{start_date},{end_date}"

        # Endpoint suffix to get all accumulations
        url_append = "?properties=accumulations"

        # Full URL
        full_url = endpt + hist_ag_url + url_append

        # Get JSON
        ag_norms_json = rq.get(full_url, headers=agro_header).json()
        
        try:
            grid_df.loc[index, 'ppet'] = ag_norms_json.get('accumulations').get('ppet')
            
        except:
            grid_df.loc[index, 'ppet'] = np.nan
            
    return grid_df

ppet_grid = get_ppet(grid, "2020-04-01", "2020-04-30")

In [ ]:
ppet_grid.plot(column='ppet')

In [ ]:
ppet_grid_vals = ppet_grid[ppet_grid.ppet.isnull() == False]

In [ ]:
ppet_grid_vals.plot(column="ppet")

In [ ]:
# Generate random values for each cell and add to grid gdf
grid['test_ppet'] = [np.random.randint(0,30) for i in range(0, grid.shape[0])]

In [ ]:
# Allow user to change threshold as daily value
daily_accumulated_ppet = 0.5

# Calculate monthly value
monthly_accumulated_ppet = daily_accumulated_ppet * 30

# Calcualte if PPET meets threshold and add Boolean flag
grid['meet_thresh'] = grid['test_ppet'].apply(lambda x: 1 if x >= monthly_accumulated_ppet else 0)

In [ ]:
# Drop all rows that do not meet threshold
droughted = grid[grid.meet_thresh == 1]

# Dissolve polygons
drought_dissolve = droughted.dissolve(by='meet_thresh')

In [ ]:
# Explode polygons so that there are unique contiguous areas
exploded = drought_dissolve.explode()

In [ ]:
# Add area column
exploded["sqkm"] = exploded['geometry'].area/ 10**6

# Drop multi-index outer level generated from dissolve
exploded = exploded.droplevel(0)

In [ ]:
# Get largest polygon
largest = exploded[exploded.sqkm == exploded.sqkm.max()]

In [ ]:
# Generate convex hull
hull_largest = largest.convex_hull

# Convert to gdf
hull = gpd.GeoDataFrame(hull_largest, crs='epsg:2019')

# Rename column '0' to 'geometry'
hull.rename(columns={0:"geometry"}, inplace=True)

In [ ]:
fig, ax = plt.subplots()


exploded.plot(ax=ax, column='sqkm')
#largest.plot(ax=ax)
#largest_envelope.plot(ax=ax, alpha=0.2)
#simple_largest.plot(ax=ax, alpha=0.4, color='orange')
hull.plot(ax=ax, alpha=0.4)

plt.show()